In [1]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, default_data_collator, \
    DataCollatorForLanguageModeling
from datasets import load_dataset
from arguments import DataTrainingArguments, ModelArguments, TrainingArguments
from collections import defaultdict
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from utils import color_idx, room_idx

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base', legacy=True)

# Initialize the data arguments
data_args = DataTrainingArguments(
    datasets='floorplan',
    data_dir=None,
    train_split='train',
    max_seq_length=512,
    max_output_seq_length=512,
    max_seq_length_eval=512,
    max_output_seq_length_eval=512,
    exp='full',
    boundary_in_where='Encoder'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [4]:
from datasets import load_dataset

floorplan_dataset = load_dataset('floorplan', data_args, split='finetune',
                                 max_input_length=data_args.max_seq_length,
                                 max_output_length=data_args.max_output_seq_length,
                                 tokenizer=tokenizer, seed=1)

In [5]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input_sentence = self.dataset[idx]['input_sentence']
        output_sentence = self.dataset[idx]['output_sentence']
        input_ids = self.tokenizer(input_sentence, max_length=self.max_length, padding='max_length', truncation=True,
                                   return_tensors='pt')['input_ids'].squeeze()
        labels = self.tokenizer(output_sentence, max_length=self.max_length, padding='max_length', truncation=True,
                                return_tensors='pt')['input_ids'].squeeze()
        return {
            'input_ids': input_ids,
            'labels': labels
        }

In [6]:
floorplan_dataset = MyDataset(floorplan_dataset, tokenizer, max_length=512)

In [7]:
training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10,
    save_total_limit=1,
    logging_steps=10,
    logging_dir='logs',
    logging_first_step=True,
    logging_strategy='steps',
    # report_to='tensorboard',
    run_name='floorplan'
)

In [8]:
optimizer = Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999), eps=1e-08)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=floorplan_dataset,
    # optimizers=(optimizer, None)
)

/Users/thebui/Developer/projects/t2f/venv/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 